In [ ]:
import pyemu
import os, glob, sys
import shutil
import numpy as np
import pandas as pd

In [ ]:
orig_ws = '../neversink_mf6' # original workspace where model was built and run
template_dir = '../run_data' # workspace where PEST++ files are created
noptmax0_dir = '../noptmax0_testing/'

In [ ]:
pst=pyemu.Pst(os.path.join(template_dir,'temp.pst'))
update_forward_run=True
run_local=True
update_all_obs = True

In [ ]:
li = pst.parameter_data.partrans=='log'
li.values

In [ ]:
report_materials = '../report_materials'
if not os.path.exists(report_materials):
    os.makedirs(report_materials)
    
pst.write_obs_summary_table(os.path.join(report_materials, 'testobs.xlsx'))

### if `update_all_obs` is True, run the get_observations.py script to get a new INS file and reset all observations in the PEST object

In [ ]:
if update_all_obs is True:
    shutil.copy2('../scripts/get_observations.py',os.path.join(template_dir,'get_observations.py'))
    shutil.copy2('../scripts/get_observations.py',os.path.join(orig_ws,'get_observations.py'))
    
    os.system('python {} {} True'.format(os.path.join(orig_ws,'get_observations.py'), orig_ws))
    [shutil.copy2(cf, os.path.join(template_dir, os.path.basename(cf))) 
        for cf in glob.glob(os.path.join(orig_ws, '*.ins'))]
    [shutil.copy2(cf, os.path.join(template_dir, os.path.basename(cf))) 
        for cf in glob.glob(os.path.join(orig_ws, 'land_*.csv'))]
    
    pst.observation_data.loc[:,:] = np.nan
    pst.observation_data.dropna(inplace=True)
    pst.add_observations(os.path.join(template_dir,'obs_mf6.dat.ins'), pst_path='.')

###  set the observation groups

In [ ]:
obs = pst.observation_data

In [ ]:
obs.obgnme = 'head'

In [ ]:
obs.loc[obs.index.str.startswith('q_'), 'obgnme'] = 'flux'

In [ ]:
obs.loc[obs.index.str.startswith('perc'), 'obgnme'] = 'budget'
obs.loc[obs.index.str.startswith('land'), 'obgnme'] = 'land_surface'


In [ ]:
obs

### Set observations

In [ ]:
set_obs = True

In [ ]:
if set_obs:
    # read in sfr; make sfr obsnme/obsval dict to map to pst observation_data
    sfr_df = pd.read_csv('../obs_data/NWIS_DV_STREAMSTATS_SITES.csv')
    sfr_df['obsnme'] = 'q_' + sfr_df['site_id'].astype(str)
    sfr_df['obsval'] = (sfr_df['Mean_Annual_Flow_cfs'] * sfr_df['Average_BFI_value']) * 2446.5755455 # convert from cfs to m^3/day
    sfr_df[['obsnme', 'obsval']]
    sfr_dict = pd.Series(sfr_df['obsval'].values,index=sfr_df['obsnme']).to_dict()
    
    # read in nwis heads; make nwis head obsnme/obsval dict
    nwis_gw_df = pd.read_csv('../obs_data/NWIS_GW_DV_data.csv')
    nwis_gw_df['obsnme'] = 'h_' + nwis_gw_df['site_no'].astype(str)
    nwis_gw_df['obsval'] = nwis_gw_df['gw_elev_m']
    nwis_gw_dict = pd.Series(nwis_gw_df['obsval'].values,index=nwis_gw_df['obsnme']).to_dict()
    
    # read in DEC heads; make DEC heads obsnme/obsval dict
    DEC_gw_df = pd.read_csv('../obs_data/NY_DEC_GW_sites.csv')
    DEC_gw_df['obsnme'] = ('h_' + DEC_gw_df['WellNO'].astype(str)).str.lower()
    DEC_gw_df['obsval'] = DEC_gw_df['gw_elev_m']
    DEC_gw_dict = pd.Series(DEC_gw_df['obsval'].values,index=DEC_gw_df['obsnme']).to_dict()
    
    # map SFR values to observation_data
    obs.loc[obs.obsnme.isin(sfr_dict.keys()), 'obsval'] = obs.obsnme.map(sfr_dict)
    
    # map nwis heads to observation_data
    obs.loc[obs.obsnme.isin(nwis_gw_dict.keys()), 'obsval'] = obs.obsnme.map(nwis_gw_dict)
    
    # map DEC heads to SRF observation_data
    obs.loc[obs.obsnme.isin(DEC_gw_dict.keys()), 'obsval'] = obs.obsnme.map(DEC_gw_dict)
    
    # set up percent discrepancy as dummy value
    obs.loc[obs.obgnme=='budget', 'obsval'] = -99999
    
    # get the land surface obs
    lsobs_df = pd.read_csv('../neversink_mf6/land_surf_obs-observations.csv', index_col=0)
    
    obs.loc[obs.obgnme=='land_surface', 'obsval'] = lsobs_df.obsval

In [ ]:
obs

### first cut at weights

In [ ]:
obs.loc[obs.obsnme=='q_1436500', 'weight'] = 3.33/obs.loc[obs.obsnme=='q_1436500'].obsval
obs.loc[obs.obsnme=='q_1366650', 'weight'] = 10/obs.loc[obs.obsnme=='q_1366650'].obsval


In [ ]:
obs.loc[obs.obgnme=='head', 'weight'] = 1/5
obs.loc[obs.obgnme=='land_surface', 'weight'] = 1/10

In [ ]:
obs.loc[obs.obgnme=='budget', 'weight'] = 0.0

In [ ]:
obs

## update some parameter bounds

In [ ]:
pars = pst.parameter_data

### K-zones set to not get too crazy high

In [ ]:
#  read in k value lookup table to df

#  original table

k_df_original = pd.read_excel(
    '../k_processing/Rondout_Neversink_GeologyLookupTable.xlsx',
    sheet_name='Sheet2'
)
k_df_original.index = k_df_original.Lookup_Code

k_df = pd.read_excel(
    '../k_processing/Rondout_Neversink_GeologyLookupTable_jhw.xlsx',
    sheet_name='Sheet2'
)

k_df.index = k_df.Lookup_Code

print('Using mean K value')
k_df['Kh_ft_d_mean'] = (k_df['Kh_ft_d_lower'] + k_df['Kh_ft_d_upper']) / 2
k_df['Kh_m_d'] = k_df['Kh_ft_d_mean'] * 0.3048
    
k_df['Kh_m_d_lower'] = k_df['Kh_ft_d_lower'] * .3048
k_df['Kh_m_d_upper'] = k_df['Kh_ft_d_upper'] * .3048

k_df['K_upper_mult'] = k_df['Kh_m_d_upper'] / k_df['Kh_m_d']
k_df['K_lower_mult'] = k_df['Kh_m_d_lower'] / k_df['Kh_m_d']


k_df

In [ ]:
k_mult_zones = [int(i.split(':')[-1]) for i in pars.loc[pars.parnme.str.startswith('multiplier_k')].index]
np.unique(k_mult_zones)

In [ ]:
upper_mults = [k_df.loc[i].K_upper_mult for i in k_mult_zones]
lower_mults = [k_df.loc[i].K_lower_mult for i in k_mult_zones]

In [ ]:
pars.loc[pars.parnme.str.startswith('multiplier_k'), 'parlbnd'] = lower_mults
pars.loc[pars.parnme.str.startswith('multiplier_k'), 'parubnd'] = upper_mults


### pilot points set to mean upper and lower bounds diffs

In [ ]:
mean_lower = k_df.K_lower_mult.mean()
mean_upper = k_df.K_upper_mult.mean()
mean_lower,mean_upper

In [ ]:
pars.loc[pars.pargp.str.startswith('k'), 'parlbnd'] = mean_lower + 0.01
pars.loc[pars.pargp.str.startswith('k'), 'parubnd'] = mean_upper - 0.01
pars.loc[pars.pargp.str.startswith('sfrk'), 'parlbnd'] = 0.1
pars.loc[pars.pargp.str.startswith('sfrk'), 'parubnd'] = 10.0
pars.loc[pars.pargp=='chd', 'partrans'] = 'fixed'

In [ ]:
parsum = pst.write_par_summary_table('../report_materials/initial_parsum.xlsx', report_in_linear_space=True)
parsum

## update the forward run to run


In [ ]:

frunlines = open(os.path.join(template_dir, 'forward_run.py'), 'r').readlines()
if update_forward_run is True and './mf6' not in ' '.join([i.strip() for i in frunlines]):
    print('updating forward_run.py')
    with open(os.path.join(template_dir, 'forward_run.py'), 'w') as ofp:
        for line in frunlines:
            if '__main__' in line:
                print('ginger')
                ofp.write("    os.system('./mf6')\n")
                ofp.write("    os.system('python get_observations.py . false')\n")
                ofp.write('{}\n'.format(line)) 
            else:
                ofp.write(line)          


In [ ]:
import sys
sys.platform

In [ ]:
pst.control_data.__dict__

### set noptmax = 0 and a couple ++ options and write out PST file

In [ ]:
pst.pestpp_options["ies_num_reals"] = 500  
pst.pestpp_options["ies_bad_phi_sigma"] = 2
pst.pestpp_options["overdue_giveup_fac"] = 4
pst.pestpp_options["ies_save_rescov"] = True
pst.pestpp_options["ies_no_noise"] = True
pst.pestpp_options["ies_drop_conflicts"] = True
pst.pestpp_options["ies_pdc_sigma_distance"] = 2.0
#pst.pestpp_options['rand_seed']=420042

In [ ]:
pst.control_data.noptmax = 0

In [ ]:
pst.write(os.path.join(template_dir,'prior_mc.pst'))

In [ ]:
if os.path.exists(noptmax0_dir):
    shutil.rmtree(noptmax0_dir)
shutil.copytree(template_dir, noptmax0_dir)

In [ ]:
pst.write_obs_summary_table('../report_materials/obs_initial.xlsx')

### For Nick -- remove backslashes from `mult2model_info.csv` for running on linux

In [ ]:
if sys.platform == 'win32':
    f = open(os.path.join(template_dir, 'mult2model_info.csv'), "r")
    lines = f.readlines()
    f.close()

    output_lines = []
    for line in lines:
        output_lines.append(line.replace('\\', "/"))

    f = open(os.path.join(template_dir, 'mult2model_info.csv'), "w")
    f.write(''.join(output_lines))
    f.close()

### and the pest file

In [ ]:
if sys.platform == 'win32':
    f = open(os.path.join(template_dir, 'prior_mc.pst'), "r")
    lines = f.readlines()
    f.close()

    output_lines = []
    for line in lines:
        output_lines.append(line.replace('\\', "/"))

    f = open(os.path.join(template_dir, 'prior_mc.pst'), "w")
    f.write(''.join(output_lines))
    f.close()